In [2]:
import numpy as np

from sklearn.preprocessing import LabelEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

import pandas as pd
import pytorch_lightning as pl

import torch
from torch import nn

import distvis

AttributeError: module 'torch.backends' has no attribute 'quantized'

In [ ]:
datos = pd.read_csv('../datos/datos_procesados.csv').iloc[:, 1:]

In [58]:
numerical_features = [
    'edad', 'meses_empleo'
]

categorical_features = [
    'sexo', 'rol_hogar', 'tipo_zona', 
    'termino_nivel', 'cine', 'est_conyugal', 'region', 'comuna', 'provincia',
    'est_subcontratado', 'categoria_empleo', 'grupo_ocupacion', 'nacionalidad'
]

features = categorical_features + numerical_features
target = "sueldo_neto"

In [59]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=np.NaN)), #dummy imputer
    ('scaler', StandardScaler())
    ])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=None, strategy='constant', fill_value=-1)),
    ('label', OrdinalEncoder(handle_unknown="return_nan")) #uknown values goes to -1
    ])

preprocessor = ColumnTransformer(               #preprocessor rearenge columns
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numerical_features)]
)

#target_scaler = StandardScaler()

preprocessor.fit(datos[features])
#target_scaler.fit(datos[target])

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value=-1,
                                                                missing_values=None,
                                                                strategy='constant')),
                                                 ('label',
                                                  OrdinalEncoder(handle_unknown='return_nan'))]),
                                 ['sexo', 'rol_hogar', 'tipo_zona',
                                  'termino_nivel', 'cine', 'est_conyugal',
                                  'region', 'comuna', 'provincia',
                                  'est_subcontratado', 'categoria_empleo',
                                  'grupo_ocupacion', 'nacionalidad']),
                                ('num',
                                 Pipeline(steps=[('imputer',
                                  

In [66]:
transform_features = preprocessor.transform(datos[features])

cat_indices = [features.index(x) for x in categorical_features]
num_indices = [features.index(x) for x in numerical_features]

x_cat = torch.IntTensor(transform_features[:, cat_indices])
x_num = torch.FloatTensor(transform_features[:, num_indices])

In [86]:
emb_szs = {feature: 5 for feature in categorical_features}
n_categories = {f: transform_features[:, features.index(f)].max().astype(int) for f in categorical_features}
n_categories

{'sexo': 2,
 'rol_hogar': 11,
 'tipo_zona': 3,
 'termino_nivel': 2,
 'cine': 10,
 'est_conyugal': 7,
 'region': 16,
 'comuna': 278,
 'provincia': 52,
 'est_subcontratado': 3,
 'categoria_empleo': 4,
 'grupo_ocupacion': 10,
 'nacionalidad': 9}

In [88]:
class TabularModel(pl.LightningModule):
    
    def __init__(self, emb_szs, n_num, layers):
        self.embeddings = {f: nn.Embedding(n_categories[f], emb_szs[f]) for f in categorical_features}
        n_emb = sum(emb_sz for emb_sz in emb_szs.values)
        sizes = [n_emb + n_num] + layers + [1]
        actns = [nn.ReLU(inplace=True) for _ in range(len(sizes)-2)] + [None]
        _layers = [LinBnDrop(sizes[i], sizes[i+1], bn=use_bn and (i!=len(actns)-1 or bn_final), p=p, act=a, lin_first=lin_first)
                       for i,(p,a) in enumerate(zip(ps+[0.],actns))]
        
    def forward(self, x_cat, x_num):
        x = [emb(x_cat[:, categorical_features.index(f)]) for f, emb in self.embeddings.items()]
        x = torch.cat(x, 1)
        x_num = self.
        
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)
        if self.n_cont != 0:
            if self.bn_cont is not None: x_cont = self.bn_cont(x_cont)
            x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont
        return self.layers(x)

In [71]:
nn.em


{5}